In [ ]:
import torch, torchvision

print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch

# 数据集的路径
path = "./DataSet"
# 使用的设备
device = torch.device("cuda")
# 数据预处理,转换为张量并归一化
transform = transforms.Compose(
    [
        transforms.ToTensor(),  # 将图像转换为张量
        transforms.Normalize((0.1307,), (0.3081,)),
    ]
)  # 归一化处理
# 将数据集使用预处理定义好的 transform
train_set = MNIST(root=path, train=True, download=True, transform=transform)
test_set = MNIST(root=path, train=False, download=True, transform=transform)
# 加载数据集并设置批次大小
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)
# 修正 print 语句，明确输出内容
print(
    f"训练集的 batch 数量: {len(train_loader)}，每个 batch 中的数据数量: {train_loader.batch_size}"
)
print(
    f"测试集的 batch 数量: {len(test_loader)}，每个 batch 中的数据数量: {test_loader.batch_size}"
)

# 图片可视化
# import matplotlib.pyplot as plt
# # 获取一个批次的数据
# images, labels = next(iter(train_loader))

# #展示图片
# plt.figure(figsize=(10, 10))
# for i in range(25):
#     plt.subplot(5, 5, i + 1)
#     # 确保图像数据形状正确
#     img = images[i].squeeze().numpy()
#     plt.imshow(img, cmap='gray')
#     plt.axis('off')
# plt.show()

训练集的 batch 数量: 938，每个 batch 中的数据数量: 64
测试集的 batch 数量: 157，每个 batch 中的数据数量: 64


In [ ]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import ReLU
from torch.nn import MaxPool2d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn.init import xavier_uniform_
from torch.nn.init import kaiming_uniform_

# 定义 MNIST_Net 类，继承自 PyTorch 的 Module 类


class MNIST_Net(Module):
    def __init__(self):
        super().__init__()
        # 卷积层 1：输入通道数为 1，输出通道数为 32，卷积核大小为 3x3
        self.conv1 = Conv2d(1, 32, (3, 3))
        # 使用 Kaiming 初始化方法初始化卷积层 1 的权重，适用于 ReLU 激活函数
        kaiming_uniform_(self.conv1.weight, nonlinearity="relu")
        self.act1 = ReLU()
        # 池化层 1：使用 2x2 的最大池化，步长为 2
        self.pool1 = MaxPool2d((2, 2), stride=(2, 2))
        # 卷积层 2：输入通道数为 32，输出通道数为 32，卷积核大小为 3x3
        self.conv2 = Conv2d(32, 32, (3, 3))
        # 使用 Kaiming 初始化方法初始化卷积层 2 的权重，适用于 ReLU 激活函数
        kaiming_uniform_(self.conv2.weight, nonlinearity="relu")
        self.act2 = ReLU()
        # 池化层 2：使用 2x2 的最大池化，步长为 2
        self.pool2 = MaxPool2d((2, 2), stride=(2, 2))
        # Flatten 层：将二维特征图展平为一维向量
        self.flatten = Flatten()
        # 全连接层 1：输入维度为计算得到的值，输出维度为 100
        self.fc1 = Linear(5 * 5 * 32, 100)
        # 使用 Kaiming 初始化方法初始化全连接层 1 的权重，适用于 ReLU 激活函数
        kaiming_uniform_(self.fc1.weight, nonlinearity="relu")
        self.act3 = ReLU()
        # 全连接层 2：输入维度为 100，输出维度为 10，对应 MNIST 的 10 个类别
        self.fc2 = Linear(100, 10)
        # 使用 Xavier 初始化方法初始化全连接层 2 的权重
        xavier_uniform_(self.fc2.weight)

    # 前向传播
    def forward(self, x):
        # 卷积层 1
        x = self.conv1(x)
        x = self.act1(x)
        # 池化层 1
        x = self.pool1(x)
        # 卷积层 2
        x = self.conv2(x)
        x = self.act2(x)
        # 池化层 2
        x = self.pool2(x)
        # 使用 Flatten 层扁平化
        x = self.flatten(x)
        # 全连接层
        x = self.fc1(x)
        x = self.act3(x)
        # 输出层
        x = self.fc2(x)
        return x

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import SGD


# 训练模型
def train_model(train_loader, model):
    # 定义损失函数和优化器
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01)
    num_epochs = 10  # 训练的轮数
    running_loss = 0.0  # 初始化 running_loss
    # 遍历epoch
    for epoch in range(num_epochs):
        # 遍历训练数据
        for i, (inputs, labels) in enumerate(train_loader):
            # 将数据移动到GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            # 梯度清零
            optimizer.zero_grad()
            # 计算模型输出
            y_hat = model(inputs)
            # 计算损失
            loss = criterion(y_hat, labels)
            # 反向传播
            loss.backward()
            # 更新参数
            optimizer.step()
            running_loss += loss.item()
        # 打印每个 epoch 的损失
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}"
        )
        # 每个 epoch 结束后重置 running_loss
        running_loss = 0.0

In [ ]:
from numpy import argmax
from numpy import vstack
from sklearn.metrics import accuracy_score
# 评估模型


def evaluate_model(test_loader, model):
    predictions, actuals = list(), list()
    for i, (inputs, labels) in enumerate(test_loader):
        # 将数据移动到GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 计算模型输出
        y_hat = model(inputs)
        # 转换为 numpy 数据类型
        y_hat = y_hat.detach().cpu().numpy()
        actual = labels.cpu().numpy()
        # 转换为类标签
        y_hat = argmax(y_hat, axis=1)
        # 为stack格式化
        actual = actual.reshape((len(actual), 1))
        y_hat = y_hat.reshape((len(y_hat), 1))
        # 存储
        predictions.append(y_hat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # 计算准确率
    acc = accuracy_score(actuals, predictions)
    return acc

In [ ]:
# 产生实例,并且将实例放入GPU
MNIST_Net_model = MNIST_Net()
MNIST_Net_model.to(device)
# 训练实例
train_model(train_loader, MNIST_Net_model)
# 评估实例
acc = evaluate_model(test_loader, MNIST_Net_model)
print('Accuracy: %.3f' % acc)